In [1]:
# Preamble
from astropy.table import Table, column
import numpy as np
import os

%matplotlib inline
import matplotlib as mpl
from matplotlib import pyplot as plt
# better-looking plots
plt.rcParams['font.family'] = 'serif'
plt.rcParams['figure.figsize'] = (10.0, 8)
plt.rcParams['font.size'] = 18
mpl.ticker.AutoLocator.default_params['nbins'] = 5
mpl.ticker.AutoLocator.default_params['prune'] = 'both'

mpl.rcParams['ps.useafm'] = True
mpl.rcParams['pdf.use14corefonts'] = True
mpl.rcParams['text.usetex'] = True

import sys
sys.path.append('../astro_codes/')

### First task is to load all of the data we require

In [2]:
gz1_data = Table.read('../fits/GZ1/gz1_matched.fits')
p_cw = gz1_data['p_cw']/(gz1_data['p_ccw']+gz1_data['p_cw'])

In [4]:
sparcfire_r = Table.read('../sparcfire_directories/r/r.galaxy_matched.fits') # sparcfire data
#sparcfire_g = Table.read('../sparcfire_directories/g/g.galaxy_matched.fits')
sparcfire_samples = Table.read('../fits/sparcfire/sparcfire_samples.fits')
g_ok = Table.read('g_vs_r/g_ok.fits')

In [5]:
in_gz1 = (gz1_data['N_spiral'] >= 5) #& (gz1_data['strong_agreement'])
good_agreement = np.any([p_cw < 0.2,p_cw > 0.8],axis=0).T

spiral_sample = np.all([sparcfire_samples['in_sparcfire'],sparcfire_samples['axial_ok'],
                        sparcfire_samples['in_mass_limit'],sparcfire_samples['in_spiral']],
                       axis=0)
                        #g_ok['ok']],axis=0)

print('{} spirals in sparcfire!'.format(spiral_sample.sum()))

gz1_spirals = np.all([in_gz1,spiral_sample],axis=0)
gz1_spirals_best = np.all([gz1_spirals,good_agreement],axis=0)
print('+ {} in GZ1'.format(gz1_spirals.sum()))
print('+ {} w. agreement > 0.8'.format(gz1_spirals_best.sum()))

3109 spirals in sparcfire!
+ 3085 in GZ1
+ 2979 w. agreement > 0.8


In [7]:
#from sklearn.metrics import matthews_corrcoef
#from scipy.stats import pearsonr
chi_gz1 = (gz1_data['chirality'] == 'Swise').astype(int)

chirality_columns = ['chirality_maj',
                     'chirality_alenWtd',
                     'chirality_wtdPangSum',
                     'chirality_longestArc']

for column in chirality_columns:
    chi_sparcfire = (sparcfire_r[column] == 'Zwise').astype(int)
    chi_agree = chi_gz1 == chi_sparcfire
    gz1_agree = chi_agree[gz1_spirals]
    gz1_agree_best = chi_agree[gz1_spirals_best]
    print('--------')
    print(column+':')
    print('{}/{} galaxies agree ({}%)'.format(gz1_agree.sum(),gz1_agree.size,
                                              np.round(gz1_agree.sum()/gz1_agree.size*100,decimals=1)))
    print('{}/{} "best" galaxies agree ({}%)'.format(gz1_agree_best.sum(),gz1_agree_best.size,
                                                     np.round(gz1_agree_best.sum()/gz1_agree_best.size*100,decimals=1)))

--------
chirality_maj:
2542/3085 galaxies agree (82.4%)
2475/2979 "best" galaxies agree (83.1%)
--------
chirality_alenWtd:
2746/3085 galaxies agree (89.0%)
2680/2979 "best" galaxies agree (90.0%)
--------
chirality_wtdPangSum:
2795/3085 galaxies agree (90.6%)
2723/2979 "best" galaxies agree (91.4%)
--------
chirality_longestArc:
2641/3085 galaxies agree (85.6%)
2577/2979 "best" galaxies agree (86.5%)
